In [36]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [37]:
load_dotenv()

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_string)

In [38]:
estados = ['al', 'ba', 'ce', 'ma', 'pb', 'pe', 'pi', 'rn', 'se']
union_queries = [f"SELECT * FROM precificacao.precificacao_{e}" for e in estados]
query = "\nUNION ALL\n".join(union_queries) + ";"

print("🔄 Executando query para unir todas as tabelas...")
try:
    df = pd.read_sql(query, engine)
    print(f"✅ Carregados {len(df):,} registros no total.")
    if 'SIGLA_UF' in df.columns:
        print("✅ Estados incluídos:", sorted(df['SIGLA_UF'].unique().tolist()))
finally:
    engine.dispose()

🔄 Executando query para unir todas as tabelas...
✅ Carregados 1,794 registros no total.
✅ Estados incluídos: ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']


In [39]:
cols_profiling = [
    "fid","CD_MUN","NM_MUN","SIGLA_UF","ckey","populacao",
    "nota_veg","nota_area","nota_relevo",
    "nota_p_q1","nota_p_q2","nota_p_q3","nota_p_q4",
    "nota_insalub","nota_insalub_2",
    "area_municip","area_georef","percent_area_georef",
    "num_imoveis","area_car_total","area_car_media",
    "perimetro_total_car","perimetro_medio_car","area_max_perim",
    "nota_total_q1","nota_total_q2","nota_total_q3","nota_total_q4",
    "nota_media","valor_mun_perim","valor_mun_area"
]

In [40]:
cols_existentes = [c for c in cols_profiling if c in df.columns]
missing = [c for c in cols_profiling if c not in df.columns]

print(f"\n📋 Colunas pedidas: {len(cols_profiling)}")
print(f"✅ Colunas encontradas: {len(cols_existentes)}")
if missing:
    print(f"⚠️ Colunas NÃO encontradas (serão ignoradas): {missing}")

df_filtered = df[cols_existentes].copy()
print(f"🧾 DataFrame final para profiling: {df_filtered.shape[0]:,} linhas x {df_filtered.shape[1]} colunas")



📋 Colunas pedidas: 31
✅ Colunas encontradas: 30
⚠️ Colunas NÃO encontradas (serão ignoradas): ['area_municip']
🧾 DataFrame final para profiling: 1,794 linhas x 30 colunas


In [41]:
print("\n📈 Gerando relatório YData Profiling (HTML). Isso pode demorar bastante...")
ydata_path = "relatorio_precificacao_ydata.html"
profile = ProfileReport(df_filtered, title="Relatório Completo - Precificação", explorative=True, minimal=False)
profile.to_file(ydata_path)
print(f"✅ YData salvo em: {os.path.abspath(ydata_path)}")



📈 Gerando relatório YData Profiling (HTML). Isso pode demorar bastante...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

  7%|▋         | 2/30 [00:00<00:00, 1991.60it/s]


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
print("\n🍰 Gerando relatório Sweetviz (com patch para NumPy>=2 se necessário)...")

# Patch simples para compatibilidade (caso numpy >= 2 tenha removido VisibleDeprecationWarning)
if not hasattr(np, "VisibleDeprecationWarning"):
    np.VisibleDeprecationWarning = DeprecationWarning

import sweetviz as sv
sweetviz_path = "relatorio_precificacao_sweetviz.html"
sweet_report = sv.analyze(df_filtered)
# open_browser=False evita abrir automaticamente
sweet_report.show_html(sweetviz_path, open_browser=False)
print(f"✅ Sweetviz salvo em: {os.path.abspath(sweetviz_path)}")



🍰 Gerando relatório Sweetviz (com patch para NumPy>=2 se necessário)...


                                             |          | [  0%]   00:00 -> (? left)

Report relatorio_precificacao_sweetviz.html was generated.
✅ Sweetviz salvo em: c:\Users\Zetta\Documents\eda_precificacao\relatorio_precificacao_sweetviz.html


In [ ]:
print("\n💡 Ativando Lux para exploração visual no notebook...")
import lux
# exibir 'df_filtered' na célula abaixo mostrará o painel Lux (se estiver em ambiente Jupyter compatível)
lux.config.default_display = "lux"

# (Opcional) converter tipos básicos para ajudar Lux e os profilers:
# - transformar colunas que são ID em string
for c in ['fid','CD_MUN','ckey']:
    if c in df_filtered.columns:
        df_filtered[c] = df_filtered[c].astype(str)

print("✅ Lux ativado. Para ver sugestões de gráficos, apenas execute uma célula com:\n\ndf_filtered\n")


In [ ]:
print("\n🎯 Feito! Relatórios gerados:")
print(" -", os.path.abspath(ydata_path))
print(" -", os.path.abspath(sweetviz_path))
print("\nSe quiser, eu já preparo uma versão 'otimizada por chunks' caso o seu kernel trave ao abrir os relatórios. Quer que eu faça isso?")

# Exibir df_filtered (opcional — você pode comentar se quiser)
df_filtered



🎯 Feito! Relatórios gerados:
 - c:\Users\Zetta\Documents\eda_precificacao\relatorio_precificacao_ydata.html
 - c:\Users\Zetta\Documents\eda_precificacao\relatorio_precificacao_sweetviz.html

Se quiser, eu já preparo uma versão 'otimizada por chunks' caso o seu kernel trave ao abrir os relatórios. Quer que eu faça isso?


,fid,CD_MUN,NM_MUN,SIGLA_UF,ckey,populacao,nota_veg,nota_area,nota_relevo,nota_p_q1,...,perimetro_total_car,perimetro_medio_car,area_max_perim,nota_total_q1,nota_total_q2,nota_total_q3,nota_total_q4,nota_media,valor_mun_perim,valor_mun_area
0,1,2708600,sao miguel dos campos,AL,sao miguel dos campos al,51783.0,2.299257,2.218568,2.5,4.546304,...,1303.05896,7.99423,508.561424,15.0,20.0,16.0,17.0,17.0,1.168883e+06,2.055331e+06
1,2,2701407,campo alegre,AL,campo alegre al,32010.0,2.135962,2.435687,2.5,4.533966,...,835.39556,5.06300,203.988644,14.0,19.0,15.0,16.0,16.0,7.493749e+05,1.919425e+06
2,3,2700102,agua branca,AL,agua branca al,19003.0,2.464780,4.905586,2.5,4.617391,...,2458.05144,1.32796,14.033310,16.0,19.0,16.0,19.0,18.0,2.204946e+06,2.903588e+06
3,4,2703205,igreja nova,AL,igreja nova al,21371.0,2.150606,7.902442,4.5,4.508174,...,6685.55773,1.14636,10.457604,21.0,26.0,22.0,23.0,23.0,5.997146e+06,2.619055e+06
4,5,2709103,taquarana,AL,taquarana al,18968.0,2.146185,7.834302,4.5,4.540257,...,1493.34330,0.92870,6.863427,21.0,26.0,22.0,23.0,23.0,1.339574e+06,9.217572e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789,65,2806107,rosario do catete,SE,rosario do catete se,9275.0,2.286546,1.002461,4.5,4.591825,...,401.76144,3.24001,83.537762,14.0,19.0,15.0,16.0,16.0,3.603921e+05,6.260449e+05
1790,66,2803906,malhador,SE,malhador se,11523.0,2.487554,5.131174,7.5,4.605976,...,951.36734,1.08356,9.343209,25.0,30.0,26.0,27.0,27.0,1.495207e+06,1.084842e+06
1791,67,2807402,tobias barreto,SE,tobias barreto se,50267.0,2.231050,5.055400,2.5,4.648618,...,9835.74106,1.51974,18.379290,16.0,20.0,17.0,18.0,18.0,8.822955e+06,6.313287e+06
1792,68,2802601,graccho cardoso,SE,graccho cardoso se,5821.0,2.127238,4.755253,4.5,4.590314,...,2445.39560,1.63135,21.177975,18.0,22.0,19.0,20.0,20.0,2.193593e+06,1.512191e+06
